# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,busselton,-33.6500,115.3333,26.73,40,43,4.33,AU,1675043877
1,1,kodiak,57.7900,-152.4072,3.73,100,100,3.60,US,1675043878
2,2,ca mau,9.1769,105.1500,24.62,55,73,4.25,VN,1675043878
3,3,vazante,-17.9869,-46.9078,20.47,91,100,0.59,BR,1675043879
4,4,carnarvon,-24.8667,113.6333,25.40,70,0,4.65,AU,1675043879


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    color = "g",
    size = "Humidity", 
    scale = 1
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
id_city_df = city_data_df.copy()
id_city_df = id_city_df[(id_city_df["Max Temp"] < 23) & (id_city_df["Max Temp"] > 18)]

# Drop any rows with null values
id_city_df = id_city_df.dropna()

# Display sample data
id_city_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,vazante,-17.9869,-46.9078,20.47,91,100,0.59,BR,1675043879
6,6,bredasdorp,-34.5322,20.0403,18.02,69,94,7.22,ZA,1675043880
12,12,beloha,-25.1667,45.0500,21.73,98,42,2.02,MG,1675043882
16,16,cape town,-33.9258,18.4232,20.21,63,0,6.26,ZA,1675043880
25,25,east london,-33.0153,27.9116,18.12,68,84,2.25,ZA,1675043889
26,26,markapur,15.7333,79.2833,21.31,90,17,2.35,IN,1675043630
28,28,san patricio,28.0170,-97.5169,22.05,92,1,2.94,US,1675043891
38,38,hermanus,-34.4187,19.2345,18.22,72,85,9.23,ZA,1675043898
46,46,beroroha,-21.6667,45.1667,22.43,96,54,0.56,MG,1675043904
53,53,les cayes,18.2000,-73.7500,22.24,73,5,1.80,HT,1675043911


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = id_city_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,3,vazante,-17.9869,-46.9078,20.47,91,100,0.59,BR,1675043879,
6,6,bredasdorp,-34.5322,20.0403,18.02,69,94,7.22,ZA,1675043880,
12,12,beloha,-25.1667,45.0500,21.73,98,42,2.02,MG,1675043882,
16,16,cape town,-33.9258,18.4232,20.21,63,0,6.26,ZA,1675043880,
25,25,east london,-33.0153,27.9116,18.12,68,84,2.25,ZA,1675043889,
...,...,...,...,...,...,...,...,...,...,...,...
573,573,hualmay,-11.0964,-77.6139,22.20,83,100,2.94,PE,1675044301,
576,576,santa cruz,-17.8000,-63.1667,22.70,94,100,1.03,BO,1675044204,
582,582,patnagarh,20.7167,83.1500,19.04,61,2,0.88,IN,1675044306,
587,587,dunedin,-45.8742,170.5036,20.11,63,84,3.49,NZ,1675044038,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel

radius = 10000

params = {
    "categories":f"accommodation.hotel",
    "limit":20,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vazante - nearest hotel: Kabana
bredasdorp - nearest hotel: Victoria Hotel
beloha - nearest hotel: No hotel found
cape town - nearest hotel: Townhouse Hotel
east london - nearest hotel: No hotel found
markapur - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
beroroha - nearest hotel: No hotel found
les cayes - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
hilo - nearest hotel: Dolphin Bay Hotel
ulladulla - nearest hotel: Hotel Marlin
port lincoln - nearest hotel: Boston Hotel
ahipara - nearest hotel: YHA Ahipara
chuy - nearest hotel: Alerces
xochitepec - nearest hotel: Hotel CACTUS
airai - nearest hotel: No hotel found
ntcheu - nearest hotel: No hotel found
viljoenskroon - nearest hotel: No hotel found
cidreira - nearest hotel: Hotel Castelo
mount gambier - nearest hotel: The Old Mount Gambier Gaol
banyo - nearest hotel: No hotel found
mar del plata - nearest hotel: N

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,3,vazante,-17.9869,-46.9078,20.47,91,100,0.59,BR,1675043879,Kabana
6,6,bredasdorp,-34.5322,20.0403,18.02,69,94,7.22,ZA,1675043880,Victoria Hotel
12,12,beloha,-25.1667,45.0500,21.73,98,42,2.02,MG,1675043882,No hotel found
16,16,cape town,-33.9258,18.4232,20.21,63,0,6.26,ZA,1675043880,Townhouse Hotel
25,25,east london,-33.0153,27.9116,18.12,68,84,2.25,ZA,1675043889,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
573,573,hualmay,-11.0964,-77.6139,22.20,83,100,2.94,PE,1675044301,Hotel Mi Choacan
576,576,santa cruz,-17.8000,-63.1667,22.70,94,100,1.03,BO,1675044204,Aviador Hotel Boutique
582,582,patnagarh,20.7167,83.1500,19.04,61,2,0.88,IN,1675044306,No hotel found
587,587,dunedin,-45.8742,170.5036,20.11,63,84,3.49,NZ,1675044038,Scenic Hotel Dunedin City


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
# Configure the map plot
map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols = "Hotel Name",
    geo = True,
    tiles = "EsriNatGeo",
    color = "City",
    size = "Humidity", 
    scale = 1
)

# Display the map
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)